In [76]:
import numpy as np
import matplotlib.pyplot as plt
import random

import numpy as np

def convolution_forward(input_data, filter_weights, stride=1, padding=0):
    """
    Forward pass of convolution operation.
    
    Args:
    - input_data: Input data (numpy array) of shape (height, width, channels).
    - filter_weights: Filter weights (numpy array) of shape (filter_height, filter_width, in_channels, out_channels).
    - stride: The stride of the convolution.
    - padding: Amount of zero-padding to apply to the input data.

    Returns:
    - output: Result of the convolution operation (numpy array).
    """
    # Extract dimensions from the input data and filters
    input_height, input_width, input_depth = input_data.shape
    filter_height, filter_width, _, num_filters = filter_weights.shape
    
    # Calculate the dimensions of the output
    output_height = (input_height - filter_height + 2 * padding) // stride + 1
    output_width = (input_width - filter_width + 2 * padding) // stride + 1
    
    # Apply padding to the input data
    padded_input = np.pad(input_data, ((padding, padding), (padding, padding), (0, 0)), 'constant')
    
    # Initialize the output feature map
    output = np.zeros((output_height, output_width, num_filters))
    
    # Perform the convolution operation
    for n in range(num_filters):
        for y in range(output_height):
            for x in range(output_width):
                for c in range(input_depth):
                    output[y, x, n] += np.sum(
                        filter_weights[:, :, c, n] * 
                        padded_input[y*stride:y*stride+filter_height, x*stride:x*stride+filter_width, c]
                    )
    return output




def convolution_backward(d_output, input_data, filter_weights, stride=1, padding=0):
    """
    Backward pass of convolution operation.
    
    Args:
    - d_output: Gradient of the loss with respect to the output of the convolution (numpy array).
    - input_data: Input data that was used for the forward pass (numpy array).
    - filter_weights: Filter weights that were used for the forward pass (numpy array).
    - stride: The stride of the convolution that was used for the forward pass.
    - padding: The padding that was used for the forward pass.

    Returns:
    - d_input: Gradient of the loss with respect to the input data (numpy array).
    - d_filter: Gradient of the loss with respect to the filter weights (numpy array).
    """
    # Extract dimensions from the input data and filters
    input_height, input_width, input_depth = input_data.shape
    filter_height, filter_width, _, num_filters = filter_weights.shape
    
    # Calculate the dimensions of the output gradient
    output_height, output_width, _ = d_output.shape
    
    # Apply padding to the input data
    padded_input = np.pad(input_data, ((padding, padding), (padding, padding), (0, 0)), 'constant')
    
    # Initialize gradients with respect to the input data and the filter weights
    d_input = np.zeros_like(padded_input)
    d_filter = np.zeros_like(filter_weights)
    
    # Perform the backward convolution operation (gradient computation)
    for n in range(num_filters):
        for y in range(output_height):
            for x in range(output_width):
                for c in range(input_depth):
                    d_input[y*stride:y*stride+filter_height, x*stride:x*stride+filter_width, c] += (
                        filter_weights[:, :, c, n] * d_output[y, x, n]
                    )
                    d_filter[:, :, c, n] += (
                        padded_input[y*stride:y*stride+filter_height, x*stride:x*stride+filter_width, c] * d_output[y, x, n]
                    )
    
    # Remove padding from the input gradient
    if padding != 0:
        d_input = d_input[padding:-padding, padding:-padding, :]
    
    return d_input, d_filter



def max_pooling_forward(input_data, pool_size, stride):
    """
    Forward pass for a max pooling layer.

    Args:
    - input_data: Input data, numpy array of shape (H, W, C).
    - pool_size: The height and width of the pooling window (HH, WW).
    - stride: The stride of the pooling window.

    Returns:
    - out: Output data, numpy array of shape (H', W', C) where H' and W' are given by
      H' = 1 + (H - HH) / stride
      W' = 1 + (W - WW) / stride
    - cache: (input_data, pool_size, stride) tuple needed for the backward pass.
    """
    H, W, C = input_data.shape
    HH, WW = pool_size
    assert (H - HH) % stride == 0, 'Invalid height'
    assert (W - WW) % stride == 0, 'Invalid width'
    H_out = 1 + (H - HH) // stride
    W_out = 1 + (W - WW) // stride

    out = np.zeros((H_out, W_out, C))
    for i in range(H_out):
        for j in range(W_out):
            for c in range(C):
                h_start = i * stride
                h_end = h_start + HH
                w_start = j * stride
                w_end = w_start + WW
                window = input_data[h_start:h_end, w_start:w_end, c]
                out[i, j, c] = np.max(window)

    cache = (input_data, pool_size, stride)
    return out, cache




def max_pooling_backward(d_out, cache):
    """
    Backward pass for a max pooling layer.

    Args:
    - d_out: Upstream derivatives.
    - cache: A tuple of (input_data, pool_size, stride) as in the max pooling forward function.

    Returns:
    - dx: Gradient with respect to the input data, numpy array of the same shape as input_data.
    """
    input_data, pool_size, stride = cache
    H, W, C = input_data.shape
    HH, WW = pool_size
    H_out, W_out, _ = d_out.shape
    dx = np.zeros_like(input_data)
    
    for i in range(H_out):
        for j in range(W_out):
            for c in range(C):
                h_start = i * stride
                h_end = h_start + HH
                w_start = j * stride
                w_end = w_start + WW
                window = input_data[h_start:h_end, w_start:w_end, c]
                m = np.max(window)
                for h in range(h_start, h_end):
                    for w in range(w_start, w_end):
                        # Only backprop to the max location
                        if input_data[h, w, c] == m:
                            dx[h, w, c] += d_out[i, j, c]
                            
    return dx

# Example input data
input_data_example = np.random.rand(10, 10, 3)
# Example filter weights for a 3x3x3 filter with 3 output channels (depth)
filter_weights_example = np.random.rand(3, 3, 3, 3)

# Perform a forward pass of the convolution operation
convolution_result = convolution_forward(input_data_example, filter_weights_example, stride=1, padding=0)
convolution_result.shape, convolution_result



((8, 8, 3),
 array([[[ 6.23102153,  5.97849868,  8.2774319 ],
         [ 5.76291318,  6.90152706,  8.97554686],
         [ 7.84905275,  6.27273653,  8.79321078],
         [ 6.63415454,  5.94527994,  8.49418768],
         [ 6.26935613,  5.00804338,  7.86861706],
         [ 5.58985008,  5.58925608,  8.1564453 ],
         [ 6.93357255,  6.52392252,  8.60852964],
         [ 7.76090617,  6.69576654,  9.21175707]],
 
        [[ 6.30527547,  5.01544705,  7.45048986],
         [ 6.20059698,  5.07665599,  7.27643135],
         [ 6.65239874,  6.35289371,  8.9498176 ],
         [ 6.71742154,  6.35476495,  9.29417174],
         [ 7.02165321,  7.3092714 ,  9.40362799],
         [ 6.77915052,  6.3095474 ,  8.30293231],
         [ 7.52795942,  7.483667  ,  9.23584657],
         [ 7.0937559 ,  7.16456002,  9.88062458]],
 
        [[ 6.63342391,  6.17837096,  8.07888655],
         [ 6.30087088,  6.78916111,  8.85940463],
         [ 7.71545893,  7.75160584, 10.27127846],
         [ 7.98509301,  7.489134

In [77]:
# Example gradients with respect to the output of the convolution
d_output_example = np.random.rand(*convolution_result.shape)

# Perform a backward pass of the convolution operation
d_input_example, d_filter_example = convolution_backward(d_output_example, input_data_example, filter_weights_example, stride=1, padding=0)
d_input_example.shape, d_filter_example.shape, d_input_example, d_filter_example

((10, 10, 3),
 (3, 3, 3, 3),
 array([[[ 0.9172697 ,  0.69791795,  1.04589888],
         [ 1.93228884,  1.31680989,  1.99374049],
         [ 2.99264658,  2.12249427,  2.49136391],
         [ 3.08652135,  1.85174568,  3.018279  ],
         [ 2.45276948,  1.84450577,  2.90478561],
         [ 2.46974516,  1.45375126,  2.35553189],
         [ 2.23775085,  1.40652051,  2.62781709],
         [ 1.73426349,  1.25211366,  2.00583582],
         [ 1.39020419,  0.8450666 ,  1.44255686],
         [ 0.89483111,  0.72550288,  0.8680537 ]],
 
        [[ 1.23225961,  1.11192047,  2.17310283],
         [ 3.18169762,  2.48668362,  3.11905973],
         [ 5.12974025,  4.22007521,  5.52003619],
         [ 4.58614806,  4.25922846,  5.27563241],
         [ 4.37498145,  4.38924602,  5.40978012],
         [ 4.26429354,  3.8420388 ,  4.72062907],
         [ 3.04446492,  3.4326441 ,  4.59125988],
         [ 3.53782628,  3.5963539 ,  4.39814541],
         [ 2.53944172,  2.18884273,  2.59181528],
         [ 1.23265

In [78]:
# Example usage of max pooling forward
pool_size = (2, 2)
stride = 2
input_data_example = np.random.rand(8, 8, 3)  # Example input data for max pooling
pooled_output, cache = max_pooling_forward(input_data_example, pool_size, stride)

pooled_output.shape, pooled_output, cache




((4, 4, 3),
 array([[[0.70832813, 0.64923498, 0.9979475 ],
         [0.78653711, 0.80812766, 0.97002429],
         [0.85811736, 0.52809188, 0.9519779 ],
         [0.87940752, 0.73927772, 0.97514233]],
 
        [[0.87375592, 0.88110357, 0.42123656],
         [0.96040457, 0.76733552, 0.67639104],
         [0.95652038, 0.58753887, 0.93907536],
         [0.6992831 , 0.91811307, 0.86620306]],
 
        [[0.93722291, 0.48689194, 0.99806369],
         [0.95235181, 0.95574065, 0.9058194 ],
         [0.98593365, 0.99530518, 0.9017432 ],
         [0.86752906, 0.90590445, 0.97379975]],
 
        [[0.91807265, 0.98663682, 0.83853951],
         [0.98791971, 0.49540435, 0.96553151],
         [0.76872692, 0.54013049, 0.93426212],
         [0.32329704, 0.79652142, 0.74460535]]]),
 (array([[[0.60492594, 0.64923498, 0.9979475 ],
          [0.41184202, 0.17023782, 0.55284087],
          [0.75573123, 0.80812766, 0.09800984],
          [0.09804098, 0.50800092, 0.97002429],
          [0.6377157 , 0.5280918

In [79]:
# Example gradients from the next layer
d_out_example = np.random.rand(*pooled_output.shape)

# Perform a backward pass of max pooling
d_input_pool_example = max_pooling_backward(d_out_example, cache)
d_input_pool_example.shape, d_input_pool_example

((8, 8, 3),
 array([[[0.        , 0.01128188, 0.17641638],
         [0.        , 0.        , 0.        ],
         [0.        , 0.50460559, 0.        ],
         [0.        , 0.        , 0.35002356],
         [0.        , 0.31058733, 0.        ],
         [0.        , 0.        , 0.95324846],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 0.        ],
         [0.75047926, 0.        , 0.        ],
         [0.74990093, 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.90537152, 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.62388931, 0.20383988, 0.85143884]],
 
        [[0.        , 0.88487656, 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.89073363],
         [0.        , 0.        , 0.418684